# Data Preprocessing

In [3]:
# load library
import numpy as np
import pandas as pd
import zipfile
import urllib.request
import io
import os
import warnings
import sys
from collections import Counter
# from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_classif
from scipy.stats import entropy
warnings.filterwarnings('ignore')

In [29]:
# load raw data
DATA_DIR = "https://raw.githubusercontent.com/sitayang-code/STAT3612_Group_Project/refs/heads/main/STAT3612_Group_Project/ehr_preprocessed_seq_by_day_cat_embedding.pkl.zip"
with urllib.request.urlopen(DATA_DIR) as response:
    with zipfile.ZipFile(io.BytesIO(response.read())) as zip_file:
        with zip_file.open("ehr_preprocessed_seq_by_day_cat_embedding.pkl", "r") as file:
            EHR = pd.read_pickle(file)
train = pd.read_csv("https://raw.githubusercontent.com/sitayang-code/STAT3612_Group_Project/refs/heads/main/STAT3612_Group_Project/train.csv")
valid = pd.read_csv("https://raw.githubusercontent.com/sitayang-code/STAT3612_Group_Project/refs/heads/main/STAT3612_Group_Project/valid.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sitayang-code/STAT3612_Group_Project/refs/heads/main/STAT3612_Group_Project/test.csv")
print(train.shape)
print(test.shape)
print(valid.shape)
print(len(EHR['feature_cols']))

(49451, 8)
(16293, 7)
(16721, 8)
171


In [32]:
# drop image-related columns
train = train.drop(train.columns[[3,5,6]], axis=1)
valid = valid.drop(valid.columns[[3,5,6]], axis=1)
test = test.drop(test.columns[[3,5,6]], axis=1)
train.head(5)

,id,subject_id,hadm_id,study_id,readmitted_within_30days
0,10869829_25238191,10869829,25238191,53096236,False
1,10869829_25238191,10869829,25238191,53621244,False
2,10869829_25238191,10869829,25238191,56056422,False
3,12347278_29852086,12347278,29852086,51319270,False
4,12347278_29852086,12347278,29852086,52234220,False


In [47]:
# extract last-day observations
train_masked = train.drop_duplicates(subset = 'id', keep = 'last')
valid_masked = valid.drop_duplicates(subset = 'id', keep = 'last')
test_masked = test.drop_duplicates(subset = 'id', keep = 'last')

print(train_masked.shape)
print(valid_masked.shape)
print(test_masked.shape)

(8234, 5)
(2788, 5)
(2741, 4)


In [48]:
# merge train and valid
train_valid = pd.concat([train, valid])
train_valid_masked = pd.concat([train_masked, valid_masked])
print(train_valid.shape)
print(train_valid_masked.shape)

(66172, 5)
(11022, 5)


In [55]:
# add a new predictor (previous number of admissions)
train_valid_masked['prev_admits'] = train_valid_masked.groupby('subject_id').cumcount()
train_masked['prev_admits'] = train_masked.groupby('subject_id').cumcount()
valid_masked['prev_admits'] = valid_masked.groupby('subject_id').cumcount()
test_masked['prev_admits'] = test_masked.groupby('subject_id').cumcount()
train_masked.head(5)

,id,subject_id,hadm_id,study_id,readmitted_within_30days,prev_admits
2,10869829_25238191,10869829,25238191,56056422,False,0
5,12347278_29852086,12347278,29852086,55687417,False,0
8,11157850_29572307,11157850,29572307,58509117,True,0
20,11958726_24320913,11958726,24320913,59423150,False,0
39,11944377_26689168,11944377,26689168,59985023,False,0


In [56]:
# extract feature column names
cat_cols = [EHR['feature_cols'][i] for i in EHR['cat_idxs']]
icd_cols = EHR['icd_cols']
cat_cols.extend(icd_cols)
num_cols = list(set(EHR['feature_cols']) - set(cat_cols)) 
#num_cols.extend(['stay_len', 'prev_admits']) 
print(cat_cols)
print(num_cols)

['gender', 'ethnicity', 'Creatinine Blood', 'Sodium Blood', 'pO2 Blood', 'Basophils Other Body Fluid', 'Basophils Pleural', 'Lactate Blood', 'Anion Gap Blood', 'Eosinophils Joint Fluid', 'Hemoglobin Blood', 'Chloride Blood', 'Eosinophils Ascites', 'pH Urine', 'Calcium, Total Blood', 'Lymphocytes Ascites', 'Eosinophils Other Body Fluid', 'Eosinophils Blood', 'Lymphocytes Blood', 'Basophils Joint Fluid', 'Hematocrit Blood', 'Potassium Blood', 'H Blood', 'Monocytes Blood', 'Eosinophils Pleural', 'Troponin T Blood', 'Neutrophils Blood', 'Bicarbonate Blood', 'Basophils Blood', 'Glucose Blood', 'Basophils Ascites', 'pH Blood', 'Platelet Count Blood', 'Lymphocytes Other Body Fluid', 'Monocytes Ascites', 'Lymphocytes Joint Fluid', 'Lymphocytes Pleural', 'pCO2 Blood', 'Y90-Y99', 'G30-G32', 'O85-O92', 'C60-C63', 'F40-F48', 'M80-M85', 'R00-R09', 'J90-J94', 'A00-A09', 'E00-E07', 'F01-F09', 'F30-F39', 'H30-H36', 'D60-D64', 'N00-N08', 'F60-F69', 'I80-I89', 'I95-I99', 'N30-N39', 'K55-K64', 'F50-F59',

In [60]:
# train_valid set
train_valid_merged_admits_all = []
train_valid_merged_admits_latest = []
train_valid_merged_admits_mean_mode = []
train_valid_merged_admits_entropy_std = []
train_valid_merged_admits_max = []  
train_valid_merged_admits_min = []  
train_valid_merged_admits_q1 = [] 
train_valid_merged_admits_q3 = []  
train_valid_merged_admits_kurtosis = []  
train_valid_merged_admits_skewness = []  
train_valid_merged_admits_iqr = []  
train_valid_merged_admits_range = []  

for _, admit in train_valid_masked.iterrows():
    num_feat_arrs = len(EHR['feat_dict'][admit['id']])
    temp_dfs = []
    for i in range(num_feat_arrs):
        feat_arr = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
        merged = admit.append(feat_arr)
        merged['obs_id'] = i + 1  
        temp_dfs.append(merged.to_frame().transpose())
        train_valid_merged_admits_all.append(merged)
        if i == num_feat_arrs - 1:
            # latest data
            feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
            train_valid_merged_admits_latest.append(admit.append(feat_arr_latest))
            # mean and mode
            temp_df = pd.concat(temp_dfs, axis=0)
            temp_df[num_cols] = temp_df[num_cols].astype(float)
            feat_arr_mean_mode = temp_df[num_cols].mean().append(temp_df[cat_cols].mode().iloc[0])
            train_valid_merged_admits_mean_mode.append(admit.append(feat_arr_mean_mode))
            # entropy and std
            feat_arr_entropy_std = temp_df[num_cols].std().append(temp_df[cat_cols].apply(lambda x: entropy(x.value_counts()), axis=0))
            train_valid_merged_admits_entropy_std.append(admit.append(feat_arr_entropy_std))
            # maximum and minimum
            feat_arr_max = temp_df[num_cols].max()
            train_valid_merged_admits_max.append(admit.append(feat_arr_max))
            feat_arr_min = temp_df[num_cols].min()
            train_valid_merged_admits_min.append(admit.append(feat_arr_min))
            # Q1 and Q3
            feat_arr_q1 = temp_df[num_cols].quantile(0.25)
            train_valid_merged_admits_q1.append(admit.append(feat_arr_q1))
            feat_arr_q3 = temp_df[num_cols].quantile(0.75)
            train_valid_merged_admits_q3.append(admit.append(feat_arr_q3))
            # kurtosis
            feat_arr_kurtosis = temp_df[num_cols].kurtosis()
            train_valid_merged_admits_kurtosis.append(admit.append(feat_arr_kurtosis))
            # skewness
            feat_arr_skewness = temp_df[num_cols].skew()
            train_valid_merged_admits_skewness.append(admit.append(feat_arr_skewness))
            # IQR
            feat_arr_iqr = temp_df[num_cols].quantile(0.75) - temp_df[num_cols].quantile(0.25)
            train_valid_merged_admits_iqr.append(admit.append(feat_arr_iqr))
            # range
            feat_arr_range = temp_df[num_cols].max() - temp_df[num_cols].min()
            train_valid_merged_admits_range.append(admit.append(feat_arr_range))
            
train_valid_merged_df_all = pd.DataFrame(train_valid_merged_admits_all)
train_valid_merged_df_latest = pd.DataFrame(train_valid_merged_admits_latest)
train_valid_merged_df_mean_mode = pd.DataFrame(train_valid_merged_admits_mean_mode)
train_valid_merged_df_entropy_std = pd.DataFrame(train_valid_merged_admits_entropy_std)
train_valid_merged_df_max = pd.DataFrame(train_valid_merged_admits_max) 
train_valid_merged_df_min = pd.DataFrame(train_valid_merged_admits_min)  
train_valid_merged_df_q1 = pd.DataFrame(train_valid_merged_admits_q1)  
train_valid_merged_df_q3 = pd.DataFrame(train_valid_merged_admits_q3)  
train_valid_merged_df_kurtosis = pd.DataFrame(train_valid_merged_admits_kurtosis)  
train_valid_merged_df_skewness = pd.DataFrame(train_valid_merged_admits_skewness)  
train_valid_merged_df_iqr = pd.DataFrame(train_valid_merged_admits_iqr)  
train_valid_merged_df_range = pd.DataFrame(train_valid_merged_admits_range)  

# test set
test_merged_admits_all = []
test_merged_admits_latest = []
test_merged_admits_mean_mode = []
test_merged_admits_entropy_std = []
test_merged_admits_max = []  
test_merged_admits_min = []  
test_merged_admits_q1 = []  
test_merged_admits_q3 = []  
test_merged_admits_kurtosis = []  
test_merged_admits_skewness = []  
test_merged_admits_iqr = []  
test_merged_admits_range = []  

for _, admit in test_masked.iterrows():
    num_feat_arrs = len(EHR['feat_dict'][admit['id']])
    temp_dfs = []
    for i in range(num_feat_arrs):
        feat_arr = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
        merged = admit.append(feat_arr)
        merged['obs_id'] = i + 1  
        temp_dfs.append(merged.to_frame().transpose())
        test_merged_admits_all.append(merged)
        if i == num_feat_arrs - 1:
            # latest data
            feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
            test_merged_admits_latest.append(admit.append(feat_arr_latest))
            # mean and mode
            temp_df = pd.concat(temp_dfs, axis=0)
            temp_df[num_cols] = temp_df[num_cols].astype(float)
            feat_arr_mean_mode = temp_df[num_cols].mean().append(temp_df[cat_cols].mode().iloc[0])
            test_merged_admits_mean_mode.append(admit.append(feat_arr_mean_mode))
            # entropy and std
            feat_arr_entropy_std = temp_df[num_cols].std().append(temp_df[cat_cols].apply(lambda x: entropy(x.value_counts()), axis=0))
            test_merged_admits_entropy_std.append(admit.append(feat_arr_entropy_std))
            # maximum and minimum
            feat_arr_max = temp_df[num_cols].max()
            test_merged_admits_max.append(admit.append(feat_arr_max))
            feat_arr_min = temp_df[num_cols].min()
            test_merged_admits_min.append(admit.append(feat_arr_min))
            # Q1 and Q3
            feat_arr_q1 = temp_df[num_cols].quantile(0.25)
            test_merged_admits_q1.append(admit.append(feat_arr_q1))
            feat_arr_q3 = temp_df[num_cols].quantile(0.75)
            test_merged_admits_q3.append(admit.append(feat_arr_q3))
            # kurtosis
            feat_arr_kurtosis = temp_df[num_cols].kurtosis()
            test_merged_admits_kurtosis.append(admit.append(feat_arr_kurtosis))
            # skewness
            feat_arr_skewness = temp_df[num_cols].skew()
            test_merged_admits_skewness.append(admit.append(feat_arr_skewness))
            # IQR
            feat_arr_iqr = temp_df[num_cols].quantile(0.75) - temp_df[num_cols].quantile(0.25)
            test_merged_admits_iqr.append(admit.append(feat_arr_iqr))
            # range
            feat_arr_range = temp_df[num_cols].max() - temp_df[num_cols].min()
            test_merged_admits_range.append(admit.append(feat_arr_range))
            
test_merged_df_all = pd.DataFrame(test_merged_admits_all)
test_merged_df_latest = pd.DataFrame(test_merged_admits_latest)
test_merged_df_mean_mode = pd.DataFrame(test_merged_admits_mean_mode)
test_merged_df_entropy_std = pd.DataFrame(test_merged_admits_entropy_std)
test_merged_df_max = pd.DataFrame(test_merged_admits_max)  
test_merged_df_min = pd.DataFrame(test_merged_admits_min)  
test_merged_df_q1 = pd.DataFrame(test_merged_admits_q1) 
test_merged_df_q3 = pd.DataFrame(test_merged_admits_q3)  
test_merged_df_kurtosis = pd.DataFrame(test_merged_admits_kurtosis)  
test_merged_df_skewness = pd.DataFrame(test_merged_admits_skewness)  
test_merged_df_iqr = pd.DataFrame(test_merged_admits_iqr)  
test_merged_df_range = pd.DataFrame(test_merged_admits_range)
# train_valid_merged_df_latest

,id,subject_id,hadm_id,study_id,readmitted_within_30days,prev_admits,age,gender,ethnicity,Y90-Y99,...,PRE-NATAL VITAMINS,ANESTHETICS,ANTIBIOTICS,ANTIHYPERGLYCEMICS,ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS,SEDATIVE/HYPNOTICS,ANTIDOTES,AUTONOMIC DRUGS,VITAMINS,BIOLOGICALS
0,10869829_25238191,10869829,25238191,56056422,False,0,52,1,6,0,...,0,0,18,4,0,3,0,0,0,0
1,12347278_29852086,12347278,29852086,55687417,False,0,62,1,6,0,...,0,1,3,0,0,0,0,0,1,0
2,11157850_29572307,11157850,29572307,58509117,True,0,51,1,5,0,...,0,0,1,1,0,9,0,2,0,0
3,11958726_24320913,11958726,24320913,59423150,False,0,75,0,5,0,...,0,1,19,3,0,1,1,1,3,0
4,11944377_26689168,11944377,26689168,59985023,False,0,45,1,5,0,...,0,0,2,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11017,12416498_27663102,12416498,27663102,57899301,False,1,69,0,6,1,...,0,1,1,0,0,0,0,0,1,0
11018,17434024_20504421,17434024,20504421,58187565,False,0,87,1,6,0,...,0,1,5,0,0,2,0,0,0,0
11019,10938464_24264222,10938464,24264222,59455286,False,0,65,1,1,0,...,0,1,0,37,0,2,0,1,0,1
11020,18772706_21847893,18772706,21847893,59629014,False,1,77,1,6,0,...,0,0,8,3,0,1,0,0,0,0


In [72]:
train_valid_merged_df_all

,id,subject_id,obs_id,hadm_id,study_id,readmitted_within_30days,prev_admits,age,gender,ethnicity,...,CARDIAC DRUGS_all,PRE-NATAL VITAMINS_all,ANESTHETICS_all,ANTIBIOTICS_all,ANTIHYPERGLYCEMICS_all,SEDATIVE/HYPNOTICS_all,ANTIDOTES_all,AUTONOMIC DRUGS_all,VITAMINS_all,BIOLOGICALS_all
0,10869829_25238191,10869829,1,25238191,56056422,False,-0.439682,-0.759733,1,6,...,0.293419,-0.012045,-0.529141,1.054840,-0.112036,0.202343,-0.061578,-0.345111,-0.436682,-0.294349
1,10869829_25238191,10869829,2,25238191,56056422,False,-0.439682,-0.759733,1,6,...,0.293419,-0.012045,-0.529141,1.054840,-0.112036,0.202343,-0.061578,-0.345111,-0.436682,-0.294349
2,10869829_25238191,10869829,3,25238191,56056422,False,-0.439682,-0.759733,1,6,...,0.293419,-0.012045,-0.529141,1.054840,-0.112036,0.202343,-0.061578,-0.345111,-0.436682,-0.294349
3,10869829_25238191,10869829,4,25238191,56056422,False,-0.439682,-0.759733,1,6,...,0.293419,-0.012045,-0.529141,1.054840,-0.112036,0.202343,-0.061578,-0.345111,-0.436682,-0.294349
4,10869829_25238191,10869829,5,25238191,56056422,False,-0.439682,-0.759733,1,6,...,0.293419,-0.012045,-0.529141,1.054840,-0.112036,0.202343,-0.061578,-0.345111,-0.436682,-0.294349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165429,13963514_24742780,13963514,2,24742780,57558196,False,-0.439682,-0.944805,1,6,...,-0.033433,-0.012045,-0.529141,-0.712301,-0.242129,-0.458801,-0.061578,-0.345111,-0.436682,-0.294349
165430,13963514_24742780,13963514,3,24742780,57558196,False,-0.439682,-0.944805,1,6,...,-0.033433,-0.012045,-0.529141,-0.712301,-0.242129,-0.458801,-0.061578,-0.345111,-0.436682,-0.294349
165431,13963514_24742780,13963514,4,24742780,57558196,False,-0.439682,-0.944805,1,6,...,-0.033433,-0.012045,-0.529141,-0.712301,-0.242129,-0.458801,-0.061578,-0.345111,-0.436682,-0.294349
165432,13963514_24742780,13963514,5,24742780,57558196,False,-0.439682,-0.944805,1,6,...,-0.033433,-0.012045,-0.529141,-0.712301,-0.242129,-0.458801,-0.061578,-0.345111,-0.436682,-0.294349


In [67]:
# standardization for numeric features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

cat_cols = [EHR['feature_cols'][i] for i in EHR['cat_idxs']]
icd_cols = EHR['icd_cols']
num_cols = list(set(EHR['feature_cols']) - set(cat_cols) - set(icd_cols)) 
num_cols.extend(['prev_admits']) 

train_valid_merged_df_all[num_cols] = scaler.fit_transform(train_valid_merged_df_all[num_cols])
train_valid_merged_df_mean_mode[num_cols] = scaler.fit_transform(train_valid_merged_df_mean_mode[num_cols])
train_valid_merged_df_latest[num_cols] = scaler.fit_transform(train_valid_merged_df_latest[num_cols])
train_valid_merged_df_entropy_std[num_cols] = scaler.fit_transform(train_valid_merged_df_entropy_std[num_cols])
train_valid_merged_df_min[num_cols] = scaler.fit_transform(train_valid_merged_df_min[num_cols])
train_valid_merged_df_max[num_cols] = scaler.fit_transform(train_valid_merged_df_max[num_cols])
train_valid_merged_df_q1[num_cols] = scaler.fit_transform(train_valid_merged_df_q1[num_cols])
train_valid_merged_df_q3[num_cols] = scaler.fit_transform(train_valid_merged_df_q3[num_cols])
train_valid_merged_df_kurtosis[num_cols] = scaler.fit_transform(train_valid_merged_df_kurtosis[num_cols])
train_valid_merged_df_skewness[num_cols] = scaler.fit_transform(train_valid_merged_df_skewness[num_cols])
train_valid_merged_df_iqr[num_cols] = scaler.fit_transform(train_valid_merged_df_iqr[num_cols])
train_valid_merged_df_range[num_cols] = scaler.fit_transform(train_valid_merged_df_range[num_cols])

test_merged_df_all[num_cols] = scaler.fit_transform(test_merged_df_all[num_cols])
test_merged_df_mean_mode[num_cols] = scaler.fit_transform(test_merged_df_mean_mode[num_cols])
test_merged_df_latest[num_cols] = scaler.fit_transform(test_merged_df_latest[num_cols])
test_merged_df_entropy_std[num_cols] = scaler.fit_transform(test_merged_df_entropy_std[num_cols])
test_merged_df_min[num_cols] = scaler.fit_transform(test_merged_df_min[num_cols])
test_merged_df_max[num_cols] = scaler.fit_transform(test_merged_df_max[num_cols])
test_merged_df_q1[num_cols] = scaler.fit_transform(test_merged_df_q1[num_cols])
test_merged_df_q3[num_cols] = scaler.fit_transform(test_merged_df_q3[num_cols])
test_merged_df_kurtosis[num_cols] = scaler.fit_transform(test_merged_df_kurtosis[num_cols])
test_merged_df_skewness[num_cols] = scaler.fit_transform(test_merged_df_skewness[num_cols])
test_merged_df_iqr[num_cols] = scaler.fit_transform(test_merged_df_iqr[num_cols])
test_merged_df_range[num_cols] = scaler.fit_transform(test_merged_df_range[num_cols])

In [68]:
# drop features with constant values in the training set
train_valid_merged_df_all.insert(2, 'obs_id', train_valid_merged_df_all.pop('obs_id'))
test_merged_df_all.insert(2, 'obs_id', test_merged_df_all.pop('obs_id'))

feat_cols_all = train_valid_merged_df_all.columns[14:].to_list()
feat_cols_latest = train_valid_merged_df_latest.columns[13:].to_list()
feat_cols_mean_mode = train_valid_merged_df_mean_mode.columns[13:].to_list()
feat_cols_entropy_std = train_valid_merged_df_entropy_std.columns[13:].to_list()
feat_cols_min = train_valid_merged_df_min.columns[13:].to_list()
feat_cols_max = train_valid_merged_df_max.columns[13:].to_list()
feat_cols_q1 = train_valid_merged_df_q1.columns[13:].to_list()
feat_cols_q3 = train_valid_merged_df_q3.columns[13:].to_list()
feat_cols_kurtosis = train_valid_merged_df_kurtosis.columns[13:].to_list()
feat_cols_skewness = train_valid_merged_df_skewness.columns[13:].to_list()
feat_cols_iqr = train_valid_merged_df_iqr.columns[13:].to_list()
feat_cols_range = train_valid_merged_df_range.columns[13:].to_list()

const_cols_all = [col for col in feat_cols_all if train_valid_merged_df_all[col].nunique() <= 1]
const_cols_latest = [col for col in feat_cols_latest if train_valid_merged_df_latest[col].nunique() <= 1]
const_cols_mean_mode = [col for col in feat_cols_mean_mode if train_valid_merged_df_mean_mode[col].nunique() <= 1]
const_cols_entropy_std = [col for col in feat_cols_entropy_std if train_valid_merged_df_entropy_std[col].nunique() <= 1]
const_cols_min = [col for col in feat_cols_min if train_valid_merged_df_min[col].nunique() <= 1]
const_cols_max = [col for col in feat_cols_max if train_valid_merged_df_max[col].nunique() <= 1]
const_cols_q1 = [col for col in feat_cols_q1 if train_valid_merged_df_q1[col].nunique() <= 1]
const_cols_q3 = [col for col in feat_cols_q3 if train_valid_merged_df_q3[col].nunique() <= 1]
const_cols_kurtosis = [col for col in feat_cols_kurtosis if train_valid_merged_df_kurtosis[col].nunique() <= 1]
const_cols_skewness = [col for col in feat_cols_skewness if train_valid_merged_df_skewness[col].nunique() <= 1]
const_cols_iqr = [col for col in feat_cols_iqr if train_valid_merged_df_iqr[col].nunique() <= 1]
const_cols_range = [col for col in feat_cols_range if train_valid_merged_df_range[col].nunique() <= 1]

train_valid_merged_df_all = train_valid_merged_df_all.drop(columns=const_cols_all, axis=1)
test_merged_df_all = test_merged_df_all.drop(columns=const_cols_all, axis=1)

train_valid_merged_df_latest = train_valid_merged_df_latest.drop(columns=const_cols_latest, axis=1)
test_merged_df_latest = test_merged_df_latest.drop(columns=const_cols_latest, axis=1)

train_valid_merged_df_mean_mode = train_valid_merged_df_mean_mode.drop(columns=const_cols_mean_mode, axis=1)
test_merged_df_mean_mode = test_merged_df_mean_mode.drop(columns=const_cols_mean_mode, axis=1)

train_valid_merged_df_entropy_std = train_valid_merged_df_entropy_std.drop(columns=const_cols_entropy_std, axis=1)
test_merged_df_entropy_std = test_merged_df_entropy_std.drop(columns=const_cols_entropy_std, axis=1)

train_valid_merged_df_min = train_valid_merged_df_min.drop(columns=const_cols_min, axis=1)
test_merged_df_min = test_merged_df_min.drop(columns=const_cols_min, axis=1)

train_valid_merged_df_max = train_valid_merged_df_max.drop(columns=const_cols_max, axis=1)
test_merged_df_max = test_merged_df_max.drop(columns=const_cols_max, axis=1)

train_valid_merged_df_q1 = train_valid_merged_df_q1.drop(columns=const_cols_q1, axis=1)
test_merged_df_q1 = test_merged_df_q1.drop(columns=const_cols_q1, axis=1)

train_valid_merged_df_q3 = train_valid_merged_df_q3.drop(columns=const_cols_q3, axis=1)
test_merged_df_q3 = test_merged_df_q3.drop(columns=const_cols_q3, axis=1)

train_valid_merged_df_kurtosis = train_valid_merged_df_kurtosis.drop(columns=const_cols_kurtosis, axis=1)
test_merged_df_kurtosis = test_merged_df_kurtosis.drop(columns=const_cols_kurtosis, axis=1)

train_valid_merged_df_skewness = train_valid_merged_df_skewness.drop(columns=const_cols_skewness, axis=1)
test_merged_df_skewness = test_merged_df_skewness.drop(columns=const_cols_skewness, axis=1)

train_valid_merged_df_iqr = train_valid_merged_df_iqr.drop(columns=const_cols_iqr, axis=1)
test_merged_df_iqr = test_merged_df_iqr.drop(columns=const_cols_iqr, axis=1)

train_valid_merged_df_range = train_valid_merged_df_range.drop(columns=const_cols_range, axis=1)
test_merged_df_range = test_merged_df_range.drop(columns=const_cols_range, axis=1)

print(train_valid_merged_df_all.shape)
print(train_valid_merged_df_latest.shape)
print(train_valid_merged_df_mean_mode.shape)
print(train_valid_merged_df_entropy_std.shape)
print(train_valid_merged_df_min.shape)
print(train_valid_merged_df_max.shape)
print(train_valid_merged_df_q1.shape)
print(train_valid_merged_df_q3.shape)
print(train_valid_merged_df_kurtosis.shape)
print(train_valid_merged_df_skewness.shape)
print(train_valid_merged_df_iqr.shape)
print(train_valid_merged_df_range.shape)

(165434, 173)
(11022, 165)
(11022, 159)
(11022, 72)
(11022, 48)
(11022, 48)
(11022, 48)
(11022, 48)
(11022, 33)
(11022, 33)
(11022, 13)
(11022, 33)


In [69]:
# add suffix
train_valid_merged_df_entropy_std.columns = list(train_valid_merged_df_entropy_std.columns[:13]) + [str(col) + '_entropy_std' for col in train_valid_merged_df_entropy_std.columns[13:]]
test_merged_df_entropy_std.columns = list(test_merged_df_entropy_std.columns[:12]) + [str(col) + '_entropy_std' for col in test_merged_df_entropy_std.columns[12:]]

train_valid_merged_df_mean_mode.columns = list(train_valid_merged_df_mean_mode.columns[:13]) + [str(col) + '_mean_mode' for col in train_valid_merged_df_mean_mode.columns[13:]]
test_merged_df_mean_mode.columns = list(test_merged_df_mean_mode.columns[:12]) + [str(col) + '_mean_mode' for col in test_merged_df_mean_mode.columns[12:]]

train_valid_merged_df_latest.columns = list(train_valid_merged_df_latest.columns[:13]) + [str(col) + '_latest' for col in train_valid_merged_df_latest.columns[13:]]
test_merged_df_latest.columns = list(test_merged_df_latest.columns[:12]) + [str(col) + '_latest' for col in test_merged_df_latest.columns[12:]]

train_valid_merged_df_all.columns = list(train_valid_merged_df_all.columns[:13]) + [str(col) + '_all' for col in train_valid_merged_df_all.columns[13:]]
test_merged_df_all.columns = list(test_merged_df_all.columns[:12]) + [str(col) + '_all' for col in test_merged_df_all.columns[12:]]

train_valid_merged_df_min.columns = list(train_valid_merged_df_min.columns[:13]) + [str(col) + '_min' for col in train_valid_merged_df_min.columns[13:]]
test_merged_df_min.columns = list(test_merged_df_min.columns[:12]) + [str(col) + '_min' for col in test_merged_df_min.columns[12:]]

train_valid_merged_df_max.columns = list(train_valid_merged_df_max.columns[:13]) + [str(col) + '_max' for col in train_valid_merged_df_max.columns[13:]]
test_merged_df_max.columns = list(test_merged_df_max.columns[:12]) + [str(col) + '_max' for col in test_merged_df_max.columns[12:]]

train_valid_merged_df_q1.columns = list(train_valid_merged_df_q1.columns[:13]) + [str(col) + '_q1' for col in train_valid_merged_df_q1.columns[13:]]
test_merged_df_q1.columns = list(test_merged_df_q1.columns[:12]) + [str(col) + '_q1' for col in test_merged_df_q1.columns[12:]]

train_valid_merged_df_q3.columns = list(train_valid_merged_df_q3.columns[:13]) + [str(col) + '_q3' for col in train_valid_merged_df_q3.columns[13:]]
test_merged_df_q3.columns = list(test_merged_df_q3.columns[:12]) + [str(col) + '_q3' for col in test_merged_df_q3.columns[12:]]

train_valid_merged_df_kurtosis.columns = list(train_valid_merged_df_kurtosis.columns[:13]) + [str(col) + '_kurtosis' for col in train_valid_merged_df_kurtosis.columns[13:]]
test_merged_df_kurtosis.columns = list(test_merged_df_kurtosis.columns[:12]) + [str(col) + '_kurtosis' for col in test_merged_df_kurtosis.columns[12:]]

train_valid_merged_df_skewness.columns = list(train_valid_merged_df_skewness.columns[:13]) + [str(col) + '_skewness' for col in train_valid_merged_df_skewness.columns[13:]]
test_merged_df_skewness.columns = list(test_merged_df_skewness.columns[:12]) + [str(col) + '_skewness' for col in test_merged_df_skewness.columns[12:]]

train_valid_merged_df_iqr.columns = list(train_valid_merged_df_iqr.columns[:13]) + [str(col) + '_iqr' for col in train_valid_merged_df_iqr.columns[13:]]
test_merged_df_iqr.columns = list(test_merged_df_iqr.columns[:12]) + [str(col) + '_iqr' for col in test_merged_df_iqr.columns[12:]]

train_valid_merged_df_range.columns = list(train_valid_merged_df_range.columns[:13]) + [str(col) + '_range' for col in train_valid_merged_df_range.columns[13:]]
test_merged_df_range.columns = list(test_merged_df_range.columns[:12]) + [str(col) + '_range' for col in test_merged_df_range.columns[12:]]

In [70]:
# write csv files
# train set
train_valid_merged_df_latest.to_csv('train_data/train_valid_latest.csv', index=False)
train_valid_merged_df_mean_mode.to_csv('train_data/train_valid_mean_mode.csv', index=False)
train_valid_merged_df_all.to_csv('train_data/train_valid_all.csv', index=False)
train_valid_merged_df_entropy_std.to_csv('train_data/train_valid_entropy_std.csv', index=False)
train_valid_merged_df_min.to_csv('train_data/train_valid_min.csv', index=False)
train_valid_merged_df_max.to_csv('train_data/train_valid_max.csv', index=False)
train_valid_merged_df_q1.to_csv('train_data/train_valid_q1.csv', index=False)
train_valid_merged_df_q3.to_csv('train_data/train_valid_q3.csv', index=False)
train_valid_merged_df_kurtosis.to_csv('train_data/train_valid_kurtosis.csv', index=False)
train_valid_merged_df_skewness.to_csv('train_data/train_valid_skewness.csv', index=False)
train_valid_merged_df_iqr.to_csv('train_data/train_valid_iqr.csv', index=False)
train_valid_merged_df_range.to_csv('train_data/train_valid_range.csv', index=False)
with zipfile.ZipFile('train_data/sequence_data.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('train_data/train_valid_all.csv')
os.remove('train_data/train_valid_all.csv')

# test set
test_merged_df_latest.to_csv('test_data/test_latest.csv', index=False)
test_merged_df_mean_mode.to_csv('test_data/test_mean_mode.csv', index=False)
test_merged_df_entropy_std.to_csv('test_data/test_entropy_std.csv', index=False)
test_merged_df_all.to_csv('test_data/test_all.csv', index=False)
test_merged_df_min.to_csv('test_data/test_min.csv', index=False)
test_merged_df_max.to_csv('test_data/test_max.csv', index=False)
test_merged_df_q1.to_csv('test_data/test_q1.csv', index=False)
test_merged_df_q3.to_csv('test_data/test_q3.csv', index=False)
test_merged_df_kurtosis.to_csv('test_data/test_kurtosis.csv', index=False)
test_merged_df_skewness.to_csv('test_data/test_skewness.csv', index=False)
test_merged_df_iqr.to_csv('test_data/test_iqr.csv', index=False)
test_merged_df_range.to_csv('test_data/test_range.csv', index=False)
with zipfile.ZipFile('test_data/sequence_data.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('test_data/test_all.csv')
os.remove('test_data/test_all.csv')

In [71]:
train_valid_merged_df_q1

,id,subject_id,hadm_id,study_id,readmitted_within_30days,prev_admits,ANTIBIOTICS,SKIN PREPS,ELECT/CALORIC/H2O,COUGH/COLD PREPARATIONS,...,DIAGNOSTIC_q1,SEDATIVE/HYPNOTICS_q1,ANTIPARASITICS_q1,CARDIOVASCULAR_q1,PRE-NATAL VITAMINS_q1,ANTIVIRALS_q1,BLOOD_q1,ANALGESICS_q1,ANTIINFECTIVES/MISCELLANEOUS_q1,THYROID PREPS_q1
0,10869829_25238191,10869829,25238191,56056422,False,-0.406147,2.165779,-0.183012,0.699604,-0.340849,...,-0.184635,0.658130,-0.017724,3.298967,-0.013472,-0.194387,1.251227,0.090293,-0.154764,-0.293484
1,12347278_29852086,12347278,29852086,55687417,False,-0.406147,-0.294207,2.744654,0.117954,-0.340849,...,-0.184635,-0.429630,-0.017724,-0.590093,-0.013472,-0.194387,-0.236906,1.394388,-0.154764,-0.293484
2,11157850_29572307,11157850,29572307,58509117,True,-0.406147,-0.622206,-0.183012,0.021012,-0.340849,...,-0.184635,2.833650,-0.017724,-0.590093,-0.013472,-0.194387,-0.236906,-0.282305,-0.154764,-0.293484
3,11958726_24320913,11958726,24320913,59423150,False,-0.406147,2.329779,2.744654,1.765963,-0.340849,...,2.706059,-0.067043,-0.017724,1.504016,-0.013472,-0.194387,0.259138,2.139585,-0.154764,-0.293484
4,11944377_26689168,11944377,26689168,59985023,False,-0.406147,-0.458207,-0.183012,-0.221342,-0.340849,...,-0.184635,-0.067043,-0.017724,-0.590093,-0.013472,-0.194387,-0.236906,4.002578,-0.154764,-0.293484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11017,12416498_27663102,12416498,27663102,57899301,False,0.968294,-0.622206,-0.183012,-0.802993,-0.340849,...,-0.184635,-0.429630,-0.017724,-0.590093,-0.013472,-0.194387,-0.236906,-0.282305,-0.154764,-0.293484
11018,17434024_20504421,17434024,20504421,58187565,False,-0.406147,0.033791,-0.183012,0.263366,-0.340849,...,-0.184635,0.295543,-0.017724,1.204858,-0.013472,1.231117,-0.236906,-0.654904,-0.154764,-0.293484
11019,10938464_24264222,10938464,24264222,59455286,False,-0.406147,-0.786205,-0.183012,0.554191,2.980844,...,-0.184635,0.295543,-0.017724,-0.290934,-0.013472,-0.194387,-0.236906,-0.654904,-0.154764,-0.293484
11020,18772706_21847893,18772706,21847893,59629014,False,0.968294,0.525788,-0.183012,-0.124401,-0.340849,...,-0.184635,-0.067043,-0.017724,0.008224,-0.013472,-0.194387,-0.236906,-0.468605,-0.154764,-0.293484
